# CPA 01 Zachary Boroda 
## Analyzing Electricty Generation Data 

Data source: https://www.eia.gov/electricity/data/state/annual_generation_state.xls 

This is a data set that for each state every year over the past 30 years shows the Megawatthours of electricity produced seperated by source. The sources are listed by type of energy and type of producer, but I will be focusing on the 'Total Electric Power Industry' numbers.

I converted this data to a CSV file 

Two questions to answer: 
1) Which state produces the most clean energy in 2020? How about in 1990? Has this state changed? 

2) Which state is the most improved? ie. had the greatest increase in renewable enegery production


In [1]:
import pandas as pd
import numpy as np